# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 11:36AM,250685,10,MTR-22-0268,"Senseonics, Incorporated",Released
1,Nov 14 2022 11:34AM,250682,10,0086167288,ISDIN Corporation,Released
2,Nov 14 2022 11:34AM,250682,10,0086167292,ISDIN Corporation,Released
3,Nov 14 2022 11:34AM,250682,10,0086167293,ISDIN Corporation,Released
4,Nov 14 2022 11:34AM,250682,10,0086167294,ISDIN Corporation,Released
5,Nov 14 2022 11:34AM,250682,10,0086167295,ISDIN Corporation,Released
6,Nov 14 2022 11:34AM,250682,10,0086167349,ISDIN Corporation,Released
7,Nov 14 2022 11:34AM,250682,10,0086167353,ISDIN Corporation,Released
8,Nov 14 2022 11:34AM,250682,10,0086167355,ISDIN Corporation,Released
9,Nov 14 2022 11:34AM,250682,10,0086167356,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,250680,Released,1
29,250681,Released,13
30,250682,Released,64
31,250684,Released,2
32,250685,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250680,NaN,1.0
250681,NaN,13.0
250682,NaN,64.0
250684,NaN,2.0
250685,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250634,0.0,2.0
250638,0.0,5.0
250649,1.0,0.0
250652,1.0,0.0
250653,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250634,0,2
250638,0,5
250649,1,0
250652,1,0
250653,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250634,0,2
1,250638,0,5
2,250649,1,0
3,250652,1,0
4,250653,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250634,,2
1,250638,,5
2,250649,1,
3,250652,1,
4,250653,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated"
1,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation
65,Nov 14 2022 11:31AM,250684,10,Emerginnova
67,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation
130,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation
143,Nov 14 2022 11:10AM,250680,10,Bio-PRF
144,Nov 14 2022 11:07AM,250676,10,ISDIN Corporation
213,Nov 14 2022 11:07AM,250678,10,Eywa Pharma Inc.
231,Nov 14 2022 11:05AM,250677,10,ISDIN Corporation
291,Nov 14 2022 10:46AM,250675,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",,1
1,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,,64
2,Nov 14 2022 11:31AM,250684,10,Emerginnova,,2
3,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation,,63
4,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation,,13
5,Nov 14 2022 11:10AM,250680,10,Bio-PRF,,1
6,Nov 14 2022 11:07AM,250676,10,ISDIN Corporation,,69
7,Nov 14 2022 11:07AM,250678,10,Eywa Pharma Inc.,13,5
8,Nov 14 2022 11:05AM,250677,10,ISDIN Corporation,,60
9,Nov 14 2022 10:46AM,250675,10,"Citieffe, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1,
1,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64,
2,Nov 14 2022 11:31AM,250684,10,Emerginnova,2,
3,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation,63,
4,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation,13,
5,Nov 14 2022 11:10AM,250680,10,Bio-PRF,1,
6,Nov 14 2022 11:07AM,250676,10,ISDIN Corporation,69,
7,Nov 14 2022 11:07AM,250678,10,Eywa Pharma Inc.,5,13
8,Nov 14 2022 11:05AM,250677,10,ISDIN Corporation,60,
9,Nov 14 2022 10:46AM,250675,10,"Citieffe, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1,
1,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64,
2,Nov 14 2022 11:31AM,250684,10,Emerginnova,2,
3,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation,63,
4,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation,13,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1.0,NaN
1,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64.0,NaN
2,Nov 14 2022 11:31AM,250684,10,Emerginnova,2.0,NaN
3,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation,63.0,NaN
4,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation,13.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1.0,0.0
1,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64.0,0.0
2,Nov 14 2022 11:31AM,250684,10,Emerginnova,2.0,0.0
3,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation,63.0,0.0
4,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation,13.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4512051,552.0,23.0
15,752007,14.0,13.0
16,250653,1.0,0.0
19,751970,4.0,2.0
21,501347,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4512051,552.0,23.0
1,15,752007,14.0,13.0
2,16,250653,1.0,0.0
3,19,751970,4.0,2.0
4,21,501347,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,552.0,23.0
1,15,14.0,13.0
2,16,1.0,0.0
3,19,4.0,2.0
4,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,552.0
1,15,Released,14.0
2,16,Released,1.0
3,19,Released,4.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Executing,23.0,13.0,0.0,2.0,0.0
Released,552.0,14.0,1.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Executing,23.0,13.0,0.0,2.0,0.0
1,Released,552.0,14.0,1.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Executing,23.0,13.0,0.0,2.0,0.0
1,Released,552.0,14.0,1.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()